In [3]:
import sys
sys.path.append('../..')
sys.path.append('../data')
sys.path.append('../../helper_code')

from sklearn.linear_model import Ridge
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error
import warnings
import numpy as np
import pandas as pd
from IPython.display import display
import copy

from helper_code.custom_kernel import *

In [15]:
# Load Data

X = pd.read_csv('../data/benzene_training_data/[Benz] ANM_X.csv')
X_inv = pd.read_csv('../data/benzene_training_data/[Benz] ANM_X_inv.csv')
X_lexi = pd.read_csv('../data/benzene_training_data/[Benz] ANM_X_lexi.csv')
X_lexi_nd = pd.read_csv('../data/benzene_training_data/[Benz] ANM_X_lexi_nd.csv')
X_sorted = pd.read_csv('../data/benzene_training_data/[Benz] ANM_X_sorted.csv')
X_coulomb = pd.read_csv('../data/benzene_training_data/[Benz] ANM_X_coulomb.csv')

X_square_eig = pd.read_csv("../data/benzene_training_data/[Benz] ANM_X_square_eig.csv")
X_inv_square_eig = pd.read_csv("../data/benzene_training_data/[Benz] ANM_X_inv_square_eig.csv")
X_lexi_square_eig = pd.read_csv("../data/benzene_training_data/[Benz] ANM_X_lexi_square_eig.csv")
X_inv_lexi_square_eig = pd.read_csv("../data/benzene_training_data/[Benz] ANM_X_lexi_inv_square_eig.csv")

inv_dist_X = pd.read_csv('../data/benzene_training_data/[Benz] inv_dist_X.csv')
inv_dist_X_lexi = pd.read_csv('../data/benzene_training_data/[Benz] inv_dist_X_lexi.csv')
inv_dist_X_lexi_nd = pd.read_csv('../data/benzene_training_data/[Benz] inv_dist_X_lexi_nd.csv')
inv_dist_X_sorted = pd.read_csv('../data/benzene_training_data/[Benz] inv_dist_X_sorted.csv')
inv_dist_X_coulomb = pd.read_csv('../data/benzene_training_data/[Benz] inv_dist_X_coulomb.csv')

rand_X = pd.read_csv('../data/benzene_training_data/[Benz] rand_X.csv')
rand_X_lexi = pd.read_csv('../data/benzene_training_data/[Benz] rand_X_lexi.csv')
rand_X_lexi_nd = pd.read_csv('../data/benzene_training_data/[Benz] rand_X_lexi_nd.csv')
rand_X_sorted = pd.read_csv('../data/benzene_training_data/[Benz] rand_X_sorted.csv')
rand_X_coulomb = pd.read_csv('../data/benzene_training_data/[Benz] rand_X_coulomb.csv')

y_energy = pd.read_csv("../data/benzene_training_data/[Benz] y_energy.csv")
y_elec = pd.read_csv("../data/benzene_training_data/[Benz] y_elec.csv")
y_delta_elec = pd.read_csv("../data/benzene_training_data/[Benz] y_delta_elec.csv")
y_delta_energy = pd.read_csv("../data/benzene_training_data/[Benz] y_delta_energy.csv")

## Baseline ##

In [20]:
# Gaussian_lexi_nd

params = {'alpha': 3.755102040816326e-08, 'gamma': 1.7346938775510206e-06, 'kernel': 'rbf'}
KRR_model = KernelRidge(**params)

k_fold = KFold(n_splits=3, shuffle=True, random_state=10)
mse_scores = cross_val_score(KRR_model, X_lexi_nd, y_delta_energy, scoring='neg_mean_squared_error', cv=k_fold)
rmse_scores = np.sqrt(-mse_scores)

# Calculate the average error across all folds
avg_rmse = rmse_scores.mean()

# Print the mean squared error for each fold
for fold, rmse in enumerate(rmse_scores):
    print(f"Fold {fold+1}: RMSE = {rmse}")

# Print the average mean squared error
print(f"Average RMSE across all folds: {avg_rmse}")

Fold 1: RMSE = 0.025826652261224
Fold 2: RMSE = 0.0643194896008388
Fold 3: RMSE = 0.026773393747848515
Average RMSE across all folds: 0.0389731785366371


In [21]:
# Gaussian_sorted

params = {'alpha': 1.5306122448979593e-9, 'gamma': 2.2857142857142856e-06, 'kernel': 'rbf'}
KRR_model = KernelRidge(**params)

k_fold = KFold(n_splits=3, shuffle=True, random_state=10)
mse_scores = cross_val_score(KRR_model, X_sorted, y_delta_energy, scoring='neg_mean_squared_error', cv=k_fold)
rmse_scores = np.sqrt(-mse_scores)

# Calculate the average error across all folds
avg_rmse = rmse_scores.mean()

# Print the mean squared error for each fold
for fold, rmse in enumerate(rmse_scores):
    print(f"Fold {fold+1}: RMSE = {rmse}")

# Print the average mean squared error
print(f"Average RMSE across all folds: {avg_rmse}")

Fold 1: RMSE = 0.16305168598114503
Fold 2: RMSE = 0.20023935344664778
Fold 3: RMSE = 0.0513143171702022
Average RMSE across all folds: 0.13820178553266502


## Extended Gaussian Kernel ##

In [18]:
X_train = X_lexi.to_numpy()
y_train = y_delta_energy.to_numpy()

params = {'gamma': 4e-10, 'epsilon': 0.0004, 'beta': 2.333333333333333e-07, 'alpha': 1e-15}
alpha = params['alpha']

similarity_matrix = create_similarity_matrix(X_train, X_train, extended_gaussian_kernel, params)
krr_model = KernelRidge(kernel='precomputed', alpha=alpha)
kf = KFold(n_splits=3, shuffle=True, random_state=42)
mse_scores = cross_val_score(krr_model, similarity_matrix, y_train, scoring='neg_mean_squared_error', cv=kf)
rmse_scores = np.sqrt(-mse_scores)
avg_rmse = rmse_scores.mean()

for fold, rmse in enumerate(rmse_scores):
    print(f"Fold {fold+1}: RMSE = {rmse}")
print(f"Average RMSE across all folds: {avg_rmse}")

Fold 1: RMSE = 0.058221697528805495
Fold 2: RMSE = 0.048039677040585295
Fold 3: RMSE = 0.021127748578803913
Average RMSE across all folds: 0.042463041049398235


## Coefficient Squared Eig ##

In [35]:
params = {'alpha': 3.346938775510204e-16, 'gamma': 2.7142857142857142e-12, 'kernel': 'rbf'}
KRR_model = KernelRidge(**params)

with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    k_fold = KFold(n_splits=2, shuffle=True, random_state=10)
    mse_scores = cross_val_score(KRR_model, X_inv_lexi_square_eig, y_delta_energy, scoring='neg_mean_squared_error', cv=k_fold)
    rmse_scores = np.sqrt(-mse_scores)
    avg_rmse = rmse_scores.mean()

for fold, rmse in enumerate(rmse_scores):
    print(f"Fold {fold+1}: RMSE = {rmse}")
print(f"Average RMSE across all folds: {avg_rmse}")

Fold 1: RMSE = 0.025261290033164404
Fold 2: RMSE = 0.025361129584760507
Average RMSE across all folds: 0.025311209808962454


In [26]:
X_train = X_inv_lexi_square_eig.to_numpy()
y_train = y_delta_energy.to_numpy()

params = {'gamma': 4e-10, 'epsilon': 0.0004, 'beta': 2.333333333333333e-07, 'alpha': 1e-15}
alpha = params['alpha']

similarity_matrix = create_similarity_matrix(X_train, X_train, extended_gaussian_kernel, params)
krr_model = KernelRidge(kernel='precomputed', alpha=alpha)
kf = KFold(n_splits=3, shuffle=True, random_state=42)
mse_scores = cross_val_score(krr_model, similarity_matrix, y_train, scoring='neg_mean_squared_error', cv=kf)
rmse_scores = np.sqrt(-mse_scores)
avg_rmse = rmse_scores.mean()

for fold, rmse in enumerate(rmse_scores):
    print(f"Fold {fold+1}: RMSE = {rmse}")
print(f"Average RMSE across all folds: {avg_rmse}")

Fold 1: RMSE = 3.7189258847626396
Fold 2: RMSE = 16.20253409700828
Fold 3: RMSE = 3.4900591142142474
Average RMSE across all folds: 7.803839698661722
